In [1]:
#import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import os
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from PIL import Image
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# yolov4 드라이브와 연결
%cd /content/drive/MyDrive/yolov4/

In [ ]:
# Darknet git repository를 yolov4 폴더에 clone 
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# darknet 드라이브와 연결
%cd /content/drive/MyDrive/yolov4/darknet

In [6]:
# eyes_data.data 파일이 새로 생성

t = open('data/eyes_data.data', 'w')

In [7]:
# mask_data.data에 기록
t.write("classes = 2\n")

12

###img 파일 분류

In [ ]:
list = os.listdir('data/images')

img_list = []
for i in list :
  if i[-3:] == 'jpg' :
    img_list.append(i)

len(img_list)

In [ ]:
# img_list 가 순서되로 되어있으니 한번 섞어주기

import random

random.shuffle(img_list)
len(img_list)

970

In [ ]:
# 이후 이미지 데이터셋을 저장할 경로 지정
path = "data/images/"

# test.txt 생성

f = open('data/test.txt', 'w')
for img in img_list[:582] :
    if img[-3:]=='jpg': #jpg가 있고 아닌게 있기 때문에, jpg만 불러오는 것
        f.write(path+img+"\n") 
f.close()

# train.txt 생성

f = open('data/train.txt', 'w')
for img in img_list[582:776] :
    if img[-3:]=='jpg':
        # 문자 입력
        f.write(path+img+"\n")
f.close()

# valid.txt 생성

f = open('data/valid.txt', 'w')
for img in img_list[776:] :
    if img[-3:]=='jpg':
        # 문자 입력
        f.write(path+img+"\n")
f.close()

In [8]:
t.write("train = data/train.txt\n")
t.write("valid = data/valid.txt\n")
t.write("test = data/test.txt\n")

21

In [ ]:
# ClassNames.names 생성

f = open('data/ClassNames.names', 'w')

# 문자 입력
# f.write("Face"+"\n")
f.write("Closed_Eye"+"\n")
f.write("Opened_Eye")

# 파일 닫기
f.close()

In [9]:
# names

t.write("names = data/ClassNames.names\n")

30

In [10]:
# backup

t.write("backup = backup/")
t.close()

다음과 같이 yolov4-custom.cgf 파일을 수정한다.

* subdivision = 16
* width = 800
* height = 1280
* max_batches = num_classes * 2000 = 3 * 2000 = 6000
* steps = max_batches * 0.8, max_batches * 0.9 = 4800, 5400
* yolo layer 마다 classes=3
* yolo layer 앞에 있는 convolutional layer 마다 filers =(num_classes + 5) * 3 = 24
* channel = 1

In [ ]:
!ls

In [ ]:
%cat cfg/yolov4-custom.cfg

In [ ]:
# Pre-Trained weights 다운

!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

In [ ]:
%cd /content/drive/MyDrive/yolov4/darknet/

In [ ]:
!ls

In [ ]:
# darknet 구축

!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

In [ ]:
!make

In [ ]:
!./darknet detector train data/eyes_data.data cfg/yolov4-custom.cfg yolov4.conv.137 -dont_show -map

In [ ]:
%cd /content/drive/MyDrive/알파코 캠퍼스 /팀플 1조 (객체인식) /data/training/088_G3/

In [ ]:
!ls

In [ ]:
!./darknet detector train data/eyes_data.data cfg/yolov4-custom.cfg backup/yolov4-custom_last.weights -dont_show -map